# 4.8: Grouping Data & Aggregating Variables

In this notebook, we will be grouping and aggregating data to derive new insights and create new variables.


In [5]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os

# Create path to data directory
path = r'C:\Users\mshhan\Documents\05-2024 Instacart Basket Analysis'

# Load the merged data from the pickle file
df_final = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge.pkl'))

# Display the first few rows of the dataframe to understand its structure
df_final.head()


,order_id,user_id,order_number,day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge_existing,product_name,aisle_id,department_id,prices,_merge,price_label,busiest_day,busiest_days_updated
0,2539329,1,1,2,8,NaN,196,1,0,both,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days


## Group by 'department_id' and aggregate mean of 'order_number'


In [6]:
# Group by 'department_id' and aggregate mean of 'order_number'
dept_order_mean = df_final.groupby('department_id').agg({'order_number': 'mean'}).reset_index()

# Display the aggregated mean
print(dept_order_mean)


    department_id  order_number
0               1     15.457687
1               2     17.277920
2               3     17.179756
3               4     17.811403
4               5     15.215751
5               6     16.439806
6               7     17.225773
7               8     15.340520
8               9     15.895474
9              10     20.197148
10             11     16.170371
11             12     15.887622
12             13     16.583304
13             14     16.759763
14             15     16.165037
15             16     17.663250
16             17     15.694469
17             18     19.310397
18             19     17.177343
19             20     16.473447
20             21     22.902379


#### Observations on Mean Order Number by Department
The first noticeable observation when analyzing the mean order number for the entire dataframe, as opposed to a subset, is that there are significantly more departments listed. Overall, the mean order numbers tend to be higher for each department when considering the entire dataset.

### Analyzing Results

The mean order number for each department_id is as follows:
- Department 4 (produce) has a mean of around 19 orders per user.
- Department 17 (household) has a mean of around 11 orders per user.


### Creating Loyalty Flag

#### Step 1: Create `max_order` Column


In [8]:
# Create 'max_order' column for each user
df_final['max_order'] = df_final.groupby('user_id')['order_number'].transform('max')

# Check the first few rows to confirm the new column
df_final.head()


,order_id,user_id,order_number,day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge_existing,product_name,aisle_id,department_id,prices,_merge,price_label,busiest_day,busiest_days_updated,max_order
0,2539329,1,1,2,8,NaN,196,1,0,both,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,10
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,10
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Slowest days,10
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,10
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,both,Mid-range product,Least busy,Slowest days,10


#### Step 2: Create `loyalty_flag` Column


In [10]:
# Create 'loyalty_flag' column based on 'max_order' values
df_final.loc[df_final['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df_final.loc[(df_final['max_order'] <= 40) & (df_final['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df_final.loc[df_final['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

# Display the frequency of each loyalty flag
print(df_final['loyalty_flag'].value_counts())


loyalty_flag
Regular customer    15890458
Loyal customer      10293434
New customer         6249138
Name: count, dtype: int64


### Analyzing Spending Habits


#### Analysis of Spending Habits by Loyalty Category
The marketing team at Instacart is interested in understanding if there are differences in the spending habits among the three types of customers identified by the loyalty flag. By analyzing the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer), we find that the average price of a product purchased by a new customer is approximately $3 higher than that of a loyal customer.

In [11]:
# Check the basic statistics of product prices for each loyalty category
price_stats = df_final.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max']}).reset_index()

# Display the price statistics
print(price_stats)



       loyalty_flag     prices              
                          mean  min      max
0    Loyal customer  10.388854  1.0  99999.0
1      New customer  13.294634  1.0  99999.0
2  Regular customer  12.496419  1.0  99999.0


### Targeting Different Types of Spenders

The marketing team aims to target different types of spenders in their campaigns by examining the prices of items people are purchasing. We created a spending flag for each user based on the average price of all their orders. Users are categorized as “Low spender” if the mean price of their purchased products is lower than 10 USD and as “High spender” if it is 10 USD or higher.


#### Creating Spending Flag


In [12]:
# Create 'spending_flag' based on mean prices
df_final['avg_price'] = df_final.groupby('user_id')['prices'].transform('mean')

df_final.loc[df_final['avg_price'] < 10, 'spending_flag'] = 'Low spender'
df_final.loc[df_final['avg_price'] >= 10, 'spending_flag'] = 'High spender'

# Display the frequency of each spending flag
print(df_final['spending_flag'].value_counts())


spending_flag
Low spender     31797576
High spender      635454
Name: count, dtype: int64


#### Determining Frequent vs. Non-Frequent Customers

To send relevant notifications within the app, such as reminders to repurchase items, Instacart wants to distinguish between frequent and non-frequent customers. We created an order frequency flag that categorizes users based on the median number of days between their orders. Users are labeled as “Non-frequent customer” if the median is greater than 20 days, “Regular customer” if it is between 10 and 20 days, and “Frequent customer” if it is 10 days or less. 

There are some rows that were not labeled, which could be due to users who have only made one order.


### Creating Order Frequency Flag


In [13]:
# Create 'order_frequency_flag' based on median 'days_since_prior_order'
df_final['median_days_since_prior'] = df_final.groupby('user_id')['days_since_prior_order'].transform('median')

df_final.loc[df_final['median_days_since_prior'] > 20, 'order_frequency_flag'] = 'Non-frequent customer'
df_final.loc[(df_final['median_days_since_prior'] > 10) & (df_final['median_days_since_prior'] <= 20), 'order_frequency_flag'] = 'Regular customer'
df_final.loc[df_final['median_days_since_prior'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

# Display the frequency of each order frequency flag
print(df_final['order_frequency_flag'].value_counts())


order_frequency_flag
Frequent customer        21576963
Regular customer          7216644
Non-frequent customer     3639423
Name: count, dtype: int64


### Export the Updated Dataframe


In [14]:
# Export the updated dataframe as a pickle file
df_final.to_pickle(os.path.join(path, '02 Data','Prepared Data', 'ords_prods_merge_updated.pkl'))
